In [203]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import pandas as pd

In [204]:
driver = webdriver.Chrome('/Users/utkarsh/PycharmProjects/chromedriver')
driver.implicitly_wait(30)

In [205]:
driver.set_page_load_timeout(10)

In [206]:
#driver.get('https://etrain.info/in?PAGE=runningHistory')
# get the search textbox
#search_field = driver.find_element_by_id("runhistrn")
#search_field2 = driver.find_element_by_id("bartrainid")
#search_field.clear()

# enter search keyword and submit
#search_field.send_keys("12557")
#search_field2.send_keys("12557")
#driver.find_element_by_css_selector('.runhisbtngo.donthide.btn.btn-primary').click()

#from running status
driver.get('https://runningstatus.in/history')
driver.execute_script("document.forms['history_form'].childNodes[3].children[0].value=12531")
driver.execute_script("document.getElementById('rdatesel').value='lastyear'")
driver.execute_script("var buttons = document.getElementsByClassName('btn btn-primary');\
Array.prototype.forEach.call(buttons, function(btn, index) \
{if(btn.innerText=='Check History')btn.click();\
});")


In [207]:
# timeout = 3
# try:
#     element_present = EC.presence_of_element_located((By.CLASS_NAME, 'main'))
#     WebDriverWait(driver, timeout).until(element_present)
# except TimeoutException:
#     print("Timed out waiting for page to load")
# finally:
#     print("Page loaded")

In [208]:
import time 
time.sleep(10)
soup_level1=BeautifulSoup(driver.page_source, 'lxml')
#print(soup_level1)


In [209]:
table_data = [[cell.text for cell in row("td")]
                         for row in soup_level1("tr")]

In [210]:
complete_date_range_data=[]
station_date_range_data=[]
each_date_data={}
each_station_data={}
each_date_count=0
date_start_count=0;
for row in soup_level1("tr"):
    for cell in row('td'):
        for abbr in cell('abbr'):
            if 'station_name' in each_station_data.keys():
                station_date_range_data.append(each_station_data)
                each_station_data={}            
            each_station_data['station_name']=abbr.text
        for small in cell('small',{'class':None}):
            each_station_data['distance_from_Source']=small.text
        for span in cell('span',{'class':'text-danger'}):
            each_station_data['avg_delayed']=span.text
        for small in cell('small'):
            if date_start_count !=0:
                each_date_count+=1
                if each_date_count == 1:
                    each_date_data['departure_delayed']=small.text
                if each_date_count == 2:
                    each_date_data['arrival_delayed']=small.text
                if each_date_count == 3:
                    each_date_data['departure_time']=small.text
                if each_date_count == 4:
                    each_date_data['arrival_time']=small.text
                    complete_date_range_data.append(each_date_data)
                    each_date_data={}
                    each_date_count=0
        for a in cell('a',{'class':'text-info'}):
            date_start_count=1
            each_date_data['Date_of_journey']=a.text
    #print(row('td').find_all('abbr'))
#print(annual_data)


In [211]:
print(annual_data)

[{'Date_of_journey': '01 Jan, 2018', 'departure_delayed': 'No Delay', 'arrival_delayed': '04 Hrs 50 Mins Late', 'departure_time': '11:35 AM', 'arrival_time': '12:40 PM'}, {'Date_of_journey': '02 Jan, 2018', 'departure_delayed': 'No Delay', 'arrival_delayed': '05 Hrs 50 Mins Late', 'departure_time': '11:35 AM', 'arrival_time': '01:40 PM'}, {'Date_of_journey': '03 Jan, 2018', 'departure_delayed': 'No Delay', 'arrival_delayed': '04 Hrs 40 Mins Late', 'departure_time': '11:35 AM', 'arrival_time': '12:30 PM'}, {'Date_of_journey': '04 Jan, 2018', 'departure_delayed': 'No Delay', 'arrival_delayed': '06 Hrs 30 Mins Late', 'departure_time': '11:35 AM', 'arrival_time': '02:20 PM'}, {'Date_of_journey': '05 Jan, 2018', 'departure_delayed': 'No Delay', 'arrival_delayed': '07 Hrs 15 Mins Late', 'departure_time': '11:35 AM', 'arrival_time': '03:05 PM'}, {'Date_of_journey': '06 Jan, 2018', 'departure_delayed': 'No Delay', 'arrival_delayed': '07 Hrs 33 Mins Late', 'departure_time': '11:35 AM', 'arrival

In [212]:
df_station= pd.DataFrame(station_date_range_data)
df_date_range=pd.DataFrame(complete_date_range_data)

In [213]:
display(df_station.head(10))
display(df_date_range.head(10))

,station_name,distance_from_Source,avg_delayed
0,Gorakhpur,0 KM's,27 Mins
1,Sahjanwa,18 KM's,18 Mins
2,Maghar,27 KM's,14 Mins
3,Khalilabad,35 KM's,28 Mins
4,Basti,65 KM's,27 Mins
5,Babhnan,95 KM's,28 Mins
6,Mankapur,126 KM's,28 Mins
7,Gonda,154 KM's,42 Mins
8,Colonelganj,183 KM's,47 Mins
9,Jarwal Road,200 KM's,52 Mins


,Date_of_journey,departure_delayed,arrival_delayed,departure_time,arrival_time
0,"05 Jan, 2018",No Delay,55 Mins Late,05:30 AM,11:55 AM
1,"06 Jan, 2018",No Delay,01 Hr 19 Mins Late,05:30 AM,12:19 PM
2,"08 Jan, 2018",No Delay,01 Hr 36 Mins Late,05:30 AM,12:36 PM
3,"09 Jan, 2018",07 Hrs 00 Min Late,11 Hrs 35 Mins Late,12:30 PM,10:35 PM
4,"10 Jan, 2018",05 Hrs 15 Mins Late,07 Hrs 40 Mins Late,10:45 AM,06:40 PM
5,"12 Jan, 2018",No Delay,01 Hr 12 Mins Late,05:30 AM,12:12 PM
6,"13 Jan, 2018",No Delay,01 Hr 00 Min Late,05:30 AM,12:00 PM
7,"14 Jan, 2018",No Delay,02 Hrs 25 Mins Late,05:30 AM,01:25 PM
8,"15 Jan, 2018",01 Hr 00 Min Late,03 Hrs 13 Mins Late,06:30 AM,02:13 PM
9,"16 Jan, 2018",No Delay,02 Hrs 03 Mins Late,05:30 AM,01:03 PM


In [214]:
#these data weekly or monthly so we can incorporate changing if any there but we need to track alert for issues